In [1]:
import pandas as pd
import numpy as np
from bson import ObjectId

import keyring
import mysql.connector

import arcgis
from arcgis.gis import GIS
import datetime as dt
import getpass
from IPython.display import HTML

from arcgis.geocoding import geocode
from arcgis.features import Feature, FeatureSet, FeatureLayerCollection, use_proximity, FeatureLayer
from arcgis.features import GeoAccessor, GeoSeriesAccessor
from arcgis.network.analysis import generate_origin_destination_cost_matrix

In [2]:
username = "Glodanale"
password = keyring.get_password("arcgis", "Glodanale")
gis = GIS("https://www.arcgis.com", "Glodanale", password)

In [3]:
def pullEIDataFromMySQL():
    connection = mysql.connector.connect(
            host = "localhost",
            port = 3306,
            user = 'root',
            database = "expandinghorizons",
            password = "dB79@dG2024!"
    )
    
    cursor = connection.cursor()
    
    query = "SELECT * FROM ei"
    cursor.execute(query)
    
    rows = cursor.fetchall()
    
    df = pd.DataFrame(rows, columns=['_id', 'FirstName', 'LastName', 'ServiceHours', 'Address1', 'City', 'State', 'Zipcode', 'County', 'Longitude', 'Latitude'])
    df["Role"] = "ei"
    
    cursor.close()
    connection.close()
    
    return df

In [4]:
def pullChildDataFromMySQL():
    connection = mysql.connector.connect(
            host = "localhost",
            port = 3306,
            user = 'root',
            database = "expandinghorizons",
            password = "dB79@dG2024!"
    )
    
    cursor = connection.cursor()
    
    query = "SELECT * FROM child"
    cursor.execute(query)
    
    rows = cursor.fetchall()
    
    df = pd.DataFrame(rows, columns=['_id', 'FirstName', 'LastName', 'ServiceHours', 'Address1', 'City', 'State', 'Zipcode', 'County', 'Longitude', 'Latitude'])
    df["Role"] = "child"
    
    cursor.close()
    connection.close()
    
    return df

In [5]:
dataEI = pullEIDataFromMySQL()
dataChild = pullChildDataFromMySQL()

In [6]:
print(dataEI)
print("-------------------------------------------------------------------------------------------------")
print(dataChild)

                         _id  FirstName    LastName  ServiceHours  \
0   65d75a2ce09e67e025e8cd30       Mary  Richardson           0.0   
1   65d75bb0e09e67e025e8cd31    Kirstin      Tipton           1.0   
2   65d75c59e09e67e025e8cd32      Kassi      Knight           0.0   
3   65d75c87e09e67e025e8cd33     Rachel      Taylor           0.0   
4   65d75cb9e09e67e025e8cd34    Brannan     Vanover           0.0   
5   65d75ce1e09e67e025e8cd35     Briana     Ailshie           0.0   
6   65d75d08e09e67e025e8cd36       Dawn     Martino           0.0   
7   65d75d4ce09e67e025e8cd37      Casey      Crumly           0.0   
8   65d75d80e09e67e025e8cd38        Kim      Risner           0.0   
9   65d75da9e09e67e025e8cd39      Lydia        Ugwu           0.0   
10  65d75dd5e09e67e025e8cd3a      Emily    Jennings           0.0   
11  65d75dfae09e67e025e8cd3b      Faith       Ihrig           0.0   
12  65d75e28e09e67e025e8cd3c  Stephanie       Cowan           0.0   

                     Address1    

In [7]:
allData = pd.concat([dataEI, dataChild], ignore_index=True)
print(allData)

                          _id FirstName    LastName  ServiceHours  \
0    65d75a2ce09e67e025e8cd30      Mary  Richardson           0.0   
1    65d75bb0e09e67e025e8cd31   Kirstin      Tipton           1.0   
2    65d75c59e09e67e025e8cd32     Kassi      Knight           0.0   
3    65d75c87e09e67e025e8cd33    Rachel      Taylor           0.0   
4    65d75cb9e09e67e025e8cd34   Brannan     Vanover           0.0   
..                        ...       ...         ...           ...   
204  65e6038938f826f29267202f     Judah     Overbay           1.0   
205  65e603f238f826f292672030    Bianca       Pogue           1.0   
206  65e6044938f826f292672031   Emorett     Woodall           1.0   
207  65e6049b38f826f292672032      Remi       Woody           0.5   
208  65ef076606e0a866af56d202  Madilynn      Phipps           1.0   

                      Address1           City State Zipcode      County  \
0     904 West Hillcrest Drive   Johnson City    TN   37604  Washington   
1        151 Valley V

In [8]:
def combine_address(row):
    return f"{row['Address1']}, {row['City']}, {row['State']} {row['Zipcode']}"

allData["Full_Address"] = allData.apply(combine_address, axis=1)
#print(allData)

featureSetDF = allData.copy()
featureSetDF = allData.drop(columns=["ServiceHours", "Address1", "City", "State", "Zipcode", "County"])
featureSetDF = featureSetDF[["_id","FirstName", "LastName", "Full_Address", "Longitude", "Latitude", "Role"]]
print(featureSetDF)

                          _id FirstName    LastName  \
0    65d75a2ce09e67e025e8cd30      Mary  Richardson   
1    65d75bb0e09e67e025e8cd31   Kirstin      Tipton   
2    65d75c59e09e67e025e8cd32     Kassi      Knight   
3    65d75c87e09e67e025e8cd33    Rachel      Taylor   
4    65d75cb9e09e67e025e8cd34   Brannan     Vanover   
..                        ...       ...         ...   
204  65e6038938f826f29267202f     Judah     Overbay   
205  65e603f238f826f292672030    Bianca       Pogue   
206  65e6044938f826f292672031   Emorett     Woodall   
207  65e6049b38f826f292672032      Remi       Woody   
208  65ef076606e0a866af56d202  Madilynn      Phipps   

                                         Full_Address  Longitude   Latitude  \
0    904 West Hillcrest Drive, Johnson City, TN 37604 -82.372108  36.319485   
1           151 Valley View Drive, Abingdon, VA 24210 -81.977836  36.722124   
2        7040 Blue Springs Parkway, Mosheim, TN 37818 -82.944595  36.197052   
3            502 Nunley

In [9]:
duplicates = featureSetDF[featureSetDF.duplicated(subset=["Longitude", "Latitude"], keep=False)]
print(duplicates)

                          _id     FirstName   LastName  \
40   65d77fdce09e67e025e8cd62         Dixie      Jones   
41   65d78068e09e67e025e8cd63        Gracie      Jones   
42   65d780cee09e67e025e8cd64      Benjamin   Keasling   
43   65d78173e09e67e025e8cd65         Henry   Keasling   
48   65d7842ee09e67e025e8cd6a        Hunter     Morton   
49   65d78493e09e67e025e8cd6b          Luke     Morton   
55   65d787f5e09e67e025e8cd71       Emersyn      Jones   
56   65d7886ae09e67e025e8cd72        Olivia      Jones   
65   65d8b679e09e67e025e8cd7b        Larson     Ketron   
66   65d8b6f8e09e67e025e8cd7c         Mason     Ketron   
67   65d8b770e09e67e025e8cd7d   Christopher      Kimes   
68   65d8b7d1e09e67e025e8cd7e        Corbin      Kimes   
69   65d8b845e09e67e025e8cd7f          Aria  McClellan   
70   65d8b8b7e09e67e025e8cd80        Aurora  McClellan   
72   65d8b9c0e09e67e025e8cd82       Charlie    Mullins   
76   65d8bc1ee09e67e025e8cd86        Armani        Arp   
80   65d8be2ae

In [10]:
uniqueFeatureSetDF = featureSetDF.copy()
uniqueFeatureSetDF = uniqueFeatureSetDF.drop_duplicates(subset=["Longitude", "Latitude"], keep="first")
uniqueFeatureSetDF = uniqueFeatureSetDF.reset_index(drop=True)
print(uniqueFeatureSetDF)

                          _id FirstName    LastName  \
0    65d75a2ce09e67e025e8cd30      Mary  Richardson   
1    65d75bb0e09e67e025e8cd31   Kirstin      Tipton   
2    65d75c59e09e67e025e8cd32     Kassi      Knight   
3    65d75c87e09e67e025e8cd33    Rachel      Taylor   
4    65d75cb9e09e67e025e8cd34   Brannan     Vanover   
..                        ...       ...         ...   
184  65e6038938f826f29267202f     Judah     Overbay   
185  65e603f238f826f292672030    Bianca       Pogue   
186  65e6044938f826f292672031   Emorett     Woodall   
187  65e6049b38f826f292672032      Remi       Woody   
188  65ef076606e0a866af56d202  Madilynn      Phipps   

                                         Full_Address  Longitude   Latitude  \
0    904 West Hillcrest Drive, Johnson City, TN 37604 -82.372108  36.319485   
1           151 Valley View Drive, Abingdon, VA 24210 -81.977836  36.722124   
2        7040 Blue Springs Parkway, Mosheim, TN 37818 -82.944595  36.197052   
3            502 Nunley

In [11]:
# Define the number of rows per chunk
chunk_size = 10

# Get the total number of rows in the original dataframe
total_rows = len(uniqueFeatureSetDF)

# Initialize an empty list to store the chunks
chunks = []

# Iterate over the rows of the original dataframe
for i in range(0, total_rows, chunk_size):
    # Get a chunk of data with the specified chunk size
    chunk = uniqueFeatureSetDF.iloc[i:i+chunk_size]
    chunk.reset_index(drop=True, inplace=True)
    # Append the chunk to the list of chunks
    chunks.append(chunk)

# Print the number of chunks created
print("Number of chunks:", len(chunks))

# Print the first few rows of each chunk
for i, chunk in enumerate(chunks):
    print(f"Chunk {i+1}:")
    print(chunk)
    print()


Number of chunks: 19
Chunk 1:
                        _id FirstName    LastName  \
0  65d75a2ce09e67e025e8cd30      Mary  Richardson   
1  65d75bb0e09e67e025e8cd31   Kirstin      Tipton   
2  65d75c59e09e67e025e8cd32     Kassi      Knight   
3  65d75c87e09e67e025e8cd33    Rachel      Taylor   
4  65d75cb9e09e67e025e8cd34   Brannan     Vanover   
5  65d75ce1e09e67e025e8cd35    Briana     Ailshie   
6  65d75d08e09e67e025e8cd36      Dawn     Martino   
7  65d75d4ce09e67e025e8cd37     Casey      Crumly   
8  65d75d80e09e67e025e8cd38       Kim      Risner   
9  65d75da9e09e67e025e8cd39     Lydia        Ugwu   

                                       Full_Address  Longitude   Latitude Role  
0  904 West Hillcrest Drive, Johnson City, TN 37604 -82.372108  36.319485   ei  
1         151 Valley View Drive, Abingdon, VA 24210 -81.977836  36.722124   ei  
2      7040 Blue Springs Parkway, Mosheim, TN 37818 -82.944595  36.197052   ei  
3          502 Nunley Drive, Johnson City, TN 37604 -82.415306

In [12]:
def buildFeatureSet(df):
    features = []

    for index, row in df.iterrows():
        attributes = {"Address": row['Full_Address'], "Status": "unconnected", "OBJECTID": row['_id'], "Role": row['Role']}
        geometry = {"x": row["Longitude"], 'y': row["Latitude"]}
        feature = {"attributes": attributes, "geometry": geometry}
        features.append(feature)
        
    location_json = {
        "features": features,
        "spatialReference": {"wkid": 4326, "lastestWkid": 4326},
        "geometryType": "esriGeometryPoint",
        "fields": [
            {"name": "OBJECTID", "type": "esriFieldTypeString", "alias": "OBJECTID", "length": "50"},
            {"name": "Address", "type": "esriFieldTypeString", "alias": "Address", "length": "100"},
            {"name": "Status", "type": "esriFieldTypeString", "alias": "Status", "length": "50"},
            {"name": "Role", "type": "esriFieldTypeString", "alias": "Role", "length": "50"}
        ]
    }
        
    feature_set = FeatureSet.from_dict(location_json)
    return feature_set

In [13]:
def createFeatureLayer(feature_set, name):
    feature_layer_item = gis.content.import_data(feature_set, title=f"Feature Layer {name}")
    feature_layer_url = feature_layer_item.url
    #print("Feature Layer URL:", feature_layer_url)
    
    return feature_layer_item

In [14]:
def analyzeFeatureLayer(feature_layer_item):
    feature_layer = feature_layer_item.layers[0]
    features = feature_layer.query()

    for feature in features:
        print(feature)
        print("\n")

In [15]:
def process(df, name):
    feature_set = buildFeatureSet(df)
    feature_layer_item = createFeatureLayer(feature_set, name)
    analyzeFeatureLayer(feature_layer_item)
    
    return feature_layer_item

In [16]:
def updateDestinations(feature_layer_item, id_value):
    feature_layer = feature_layer_item.layers[0]
    feature_layer_query = feature_layer.query(where=f"OBJECTID = '{id_value}'")
    for feature in feature_layer_query.features:
        feature.attributes["Status"] = "connected"
    feature_layer.edit_features(updates=feature_layer_query.features)

In [17]:
def buildMatrix(origin, destination, name):
    items = gis.content.search(query="OD Cost Matrix", item_type="Feature Service")
    for item in items:
        item.delete()

    result = use_proximity.connect_origins_to_destinations(origins_layer=origin,
                                                            destinations_layer=destination,
                                                            origins_layer_route_id_field= "Status",
                                                            destinations_layer_route_id_field="Status",
                                                            output_name=f"OD Cost Matrix {name}",
                                                            gis=gis)

    print("I completed the function")

    od_cost_matrix_sublayer = FeatureLayer.fromitem(result, layer_id=0)
    od_df = od_cost_matrix_sublayer.query(where='1=1', as_df=True)
    myODMatrix = od_df[['From_OBJECTID','From_Address','From_Role','To_OBJECTID','To_Address','To_Role','Total_Miles','Total_Minutes']]
    print(myODMatrix)

    return myODMatrix

In [18]:
def child_child_entry(layerID, loopID, cursor, miles=0.0, minutes=0.0):
    query = ("SELECT * FROM locationdata "
             f"WHERE child_id_1 = '{layerID}' and child_id_2 = '{loopID}' "
             f"OR child_id_1 = '{loopID}' and child_id_2 = '{layerID}'" )     

    cursor.execute(query)
    rows = cursor.fetchall()

    if rows:
        updateQuery = ("UPDATE locationdata " 
                       f"SET driving_distance = {miles}, driving_time = {minutes} " 
                       f"WHERE child_id_1 = '{layerID}' and child_id_2 = '{loopID}' "
                       f"OR child_id_1 = '{loopID}' and child_id_2 = '{layerID}'" )

        cursor.execute(updateQuery)
        print(f"{layerID} to {loopID} is updated in the MySQL database")
    else:
        insertQuery = ("INSERT INTO locationdata(child_id_1, child_id_2, ei_id_1, ei_id_2, assigned, driving_distance, driving_time) "
                       "VALUES (%s, %s, %s, %s, %s, %s, %s) ")
        cursor.execute(insertQuery, (
            layerID, loopID, None, None, 0, miles, minutes))
        print(f"{layerID} to {loopID} is inserted in the MySQL database")

In [19]:
def child_ei_entry(layerID, loopID, cursor, miles=0.0, minutes=0.0):
    query = ("SELECT * FROM locationdata "
             f"WHERE child_id_1 = '{layerID}' and ei_id_1 = '{loopID}'")

    cursor.execute(query)
    rows = cursor.fetchall()

    if rows:
        updateQuery = ("UPDATE locationdata " 
                       f"SET driving_distance = {miles}, driving_time = {minutes} " 
                       f"WHERE child_id_1 = '{layerID}' and ei_id_1 = '{loopID}'")

        cursor.execute(updateQuery)
        print(f"{layerID} to {loopID} is updated in the MySQL database")
    else:
        insertQuery = ("INSERT INTO locationdata(child_id_1, child_id_2, ei_id_1, ei_id_2, assigned, driving_distance, driving_time) "
                       "VALUES (%s, %s, %s, %s, %s, %s, %s) ")
        cursor.execute(insertQuery, (
            layerID, None, loopID, None, 0, miles, minutes))
        print(f"{layerID} to {loopID} is inserted in the MySQL database")

In [20]:
def ei_child_entry(layerID, loopID, cursor, miles=0.0, minutes=0.0):
    query = ("SELECT * FROM locationdata "
             f"WHERE ei_id_1 = '{layerID}' and child_id_1 = '{loopID}'")
    print(query)

    cursor.execute(query)
    rows = cursor.fetchall()

    if rows:
        updateQuery = ("UPDATE locationdata " 
                       f"SET driving_distance = {miles}, driving_time = {minutes} " 
                       f"WHERE ei_id_1 = '{layerID}' and child_id_1 = '{loopID}'")

        cursor.execute(updateQuery)
        print(f"{layerID} to {loopID} is updated in the MySQL database")
    else:
        insertQuery = ("INSERT INTO locationdata(child_id_1, child_id_2, ei_id_1, ei_id_2, assigned, driving_distance, driving_time) "
                       "VALUES (%s, %s, %s, %s, %s, %s, %s) ")
        cursor.execute(insertQuery, (
            loopID, None, layerID, None, 0, miles, minutes))
        print(f"{layerID} to {loopID} is inserted in the MySQL database")

In [21]:
def ei_ei_entry(layerID, loopID, cursor, miles=0.0, minutes=0.0):
    query = ("SELECT * FROM locationdata "
             f"WHERE ei_id_1 = '{layerID}' and ei_id_2 = '{loopID}' "
             f"OR ei_id_1 = '{loopID}' and ei_id_2 = '{layerID}'")

    cursor.execute(query)
    rows = cursor.fetchall()

    if rows:
        updateQuery = ("UPDATE locationdata " 
                       f"SET driving_distance = {miles}, driving_time = {minutes} " 
                       f"WHERE ei_id_1 = '{layerID}' and ei_id_2 = '{loopID}' "
                       f"OR ei_id_1 = '{loopID}' and ei_id_2 = '{layerID}'")

        cursor.execute(updateQuery)
        print(f"{layerID} to {loopID} is updated in the MySQL database")

    else:
        insertQuery = ("INSERT INTO locationdata(child_id_1, child_id_2, ei_id_1, ei_id_2, assigned, driving_distance, driving_time) "
                       "VALUES (%s, %s, %s, %s, %s, %s, %s) ")
        cursor.execute(insertQuery, (
            None, None, layerID, loopID, 0, miles, minutes))
        print(f"{layerID} to {loopID} is inserted in the MySQL database")

In [22]:
def pushLocationDataToMySQL(locationDF, duplicates):
    if locationDF.empty:
        print("Error: locationDF is empty")
    else:
        connection = mysql.connector.connect(
            host = "localhost",
            port = 3306,
            user = 'root',
            database = "expandinghorizons",
            password = "dB79@dG2024!"
        )
    
        cursor = connection.cursor()
    
        if cursor:
            print(f"\n\nDatabase connected\n\n")
        else:
            print("not connected")
    
        for index, row in locationDF.iterrows():
            print()
            if row.isnull().any():
                print(f"{row['From_OBJECTID']} to {row['To_OBJECTID']} contains null values")
            else:
                fromRole = row["From_Role"]
                toRole = row["To_Role"]
                miles = row['Total_Miles']
                minutes = row['Total_Minutes']
                fromID = row["From_OBJECTID"]
                toID = row["To_OBJECTID"]
                
                print(f"fromRole: {fromRole}   toRole:{toRole}")
                
                #make sure duplicates are entered in locationdata with 0 distance and 0 drive time
                if fromID in duplicates["_id"].values:
                    print("duplicate in fromID found")
                    from_filtered_ids = duplicates.loc[duplicates["Full_Address"] == row["From_Address"], "_id"].tolist()
                    from_filtered_roles = duplicates.loc[duplicates["Full_Address"] == row["From_Address"], "Role"].tolist()
                    for a in range(0, len(from_filtered_ids)):
                        loopID = from_filtered_ids[a]
                        loopRole = from_filtered_roles[a]
                        if fromID != loopID:
                            if fromRole == "child":
                                if loopRole == "child":
                                    child_child_entry(fromID, loopID, cursor)
                                elif loopRole == "ei":
                                    child_ei_entry(fromID, loopID, cursor)
                                else:
                                    print("Error with types")
                            elif fromRole == "ei":
                                if loopRole == "child":
                                    ei_child_entry(fromID, loopID, cursor)
                                elif loopRole == "ei":
                                    ei_ei_entry(fromID, loopID, cursor)
                                else:
                                    print("Error with types")
                            else:
                                print("Error finding role")
                        else:
                            print(f"IDs match: fromID={fromID}   loopID={loopID}")
                    #exit from_filtered_ids loop
                else:
                    from_filtered_ids = [fromID]
                    from_filtered_roles = [fromRole]
                         
                if toID in duplicates["_id"].values:
                    print("duplicate in toID found")
                    to_filtered_ids = duplicates.loc[duplicates["Full_Address"] == row["To_Address"], "_id"].tolist()
                    to_filtered_roles = duplicates.loc[duplicates["Full_Address"] == row["To_Address"], "Role"].tolist()
                    for d in range(0, len(to_filtered_ids)):
                        loopID = to_filtered_ids[d]
                        loopRole = to_filtered_roles[d]
                        if toID != loopID:
                            if toRole == "child":
                                if loopRole == "child":
                                    child_child_entry(toID, loopID, cursor)
                                elif loopRole == "ei":
                                    child_ei_entry(toID, loopID, cursor)
                                else:
                                    print("Error with types")
                            elif toRole == "ei":
                                if loopRole == "child":
                                    ei_child_entry(toID, loopID, cursor)
                                elif loopRole == "ei":
                                    ei_ei_entry(toID, loopID, cursor)
                                else:
                                    print("Error with types")
                            else:
                                print("Error finding role")
                        else:
                            print(f"IDs match: loopID={loopID}   toID={toID}")
                else:
                    to_filtered_ids = [toID]
                    to_filtered_roles = [toRole]
                #end of duplicate checking    
                
                #match all from_locations with all to_locations
                for i in range(0, len(from_filtered_ids)):
                    for j in range(0, len(to_filtered_ids)):
                        fromID = from_filtered_ids[i]
                        fromRole = from_filtered_roles[i]
                        toID = to_filtered_ids[j]
                        toRole = to_filtered_roles[j]
                    
                        if fromRole == "child":
                            if toRole == "child":
                                child_child_entry(fromID, toID, cursor, miles=miles, minutes=minutes)
                            elif toRole == "ei":
                                child_ei_entry(fromID, toID, cursor, miles=miles, minutes=minutes)
                            else:
                                print("Error recieving to_role")
                        elif fromRole == "ei":
                            if toRole == "child":
                                ei_child_entry(fromID, toID, cursor, miles=miles, minutes=minutes)                                    
                            elif toRole == "ei":
                                ei_ei_entry(fromID, toID, cursor, miles=miles, minutes=minutes)                                
                            else:
                                print("Error recieving to_role")
                        else:
                            print("Error recieving from_role")
                #end of all matches
 
        connection.commit()
        cursor.close()
        connection.close()
        
        print("---------------------------------------------------------------------------------------------------------------------------------")
        print("All data entered into database")

In [36]:
def buildData(df, uniqueFeatureSetDF, duplicates): 
    newUniqueFeatureSetDF = uniqueFeatureSetDF.copy()
    for index, row in df.iterrows():        
        row_df = pd.DataFrame(row).transpose()
        row_df.reset_index(drop=True, inplace=True)
        if len(row_df) != 1:
            print("Error in df length")
        else:
            id_value = row_df.loc[0, "_id"]
            newUniqueFeatureSetDF = newUniqueFeatureSetDF[newUniqueFeatureSetDF['_id'] != id_value]
            newUniqueFeatureSetDF.reset_index(drop=True, inplace=True)
            destination = process(newUniqueFeatureSetDF, 'destination')
            originLayer = process(row_df, id_value)
            originDestinationDF = buildMatrix(originLayer, destination, id_value)
            pushLocationDataToMySQL(originDestinationDF, duplicates)
            
            print("//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////")
            print("//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////")
            print("//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////")
    print("\n\n")
    print("Complete Function")
    
    return newUniqueFeatureSetDF

In [24]:
uniqueDataRemaining = buildData(chunks[0], uniqueFeatureSetDF, duplicates)

//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
//////////////////////////////////////////////////////////////////////////////////////////////

In [25]:
uniqueDataRemaining = buildData(chunks[1], uniqueDataRemaining, duplicates)

//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
//////////////////////////////////////////////////////////////////////////////////////////////

In [26]:
uniqueDataRemaining = buildData(chunks[2], uniqueDataRemaining, duplicates)

//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
//////////////////////////////////////////////////////////////////////////////////////////////

In [27]:
uniqueDataRemaining = buildData(chunks[3], uniqueDataRemaining, duplicates)

//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
//////////////////////////////////////////////////////////////////////////////////////////////

In [28]:
uniqueDataRemaining = buildData(chunks[4], uniqueDataRemaining, duplicates)

//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
//////////////////////////////////////////////////////////////////////////////////////////////

In [29]:
uniqueDataRemaining = buildData(chunks[5], uniqueDataRemaining, duplicates)

//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
//////////////////////////////////////////////////////////////////////////////////////////////

In [30]:
uniqueDataRemaining = buildData(chunks[6], uniqueDataRemaining, duplicates)

//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
//////////////////////////////////////////////////////////////////////////////////////////////

In [31]:
uniqueDataRemaining = buildData(chunks[7], uniqueDataRemaining, duplicates)

//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
//////////////////////////////////////////////////////////////////////////////////////////////

In [32]:
uniqueDataRemaining = buildData(chunks[8], uniqueDataRemaining, duplicates)

//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
//////////////////////////////////////////////////////////////////////////////////////////////

In [33]:
uniqueDataRemaining = buildData(chunks[9], uniqueDataRemaining, duplicates)

//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
//////////////////////////////////////////////////////////////////////////////////////////////

In [37]:
uniqueDataRemaining = buildData(chunks[10], uniqueDataRemaining, duplicates)

{"geometry": {"x": -9179315.083063988, "y": 4339505.633918449, "spatialReference": {"wkid": 102100, "latestWkid": 3857}}, "attributes": {"FID": 1, "Address": "18 Glory Court, Jonesborough, TN 37659", "Status": "unconnected", "OBJECTID": "65d8d0bee09e67e025e8cdaa", "Role": "child"}}


{"geometry": {"x": -9148950.048927544, "y": 4381574.3407135, "spatialReference": {"wkid": 102100, "latestWkid": 3857}}, "attributes": {"FID": 2, "Address": "711 7th Avenue, Bristol, TN 37620", "Status": "unconnected", "OBJECTID": "65d8d17be09e67e025e8cdab", "Role": "child"}}


{"geometry": {"x": -9151330.63770847, "y": 4381840.357955383, "spatialReference": {"wkid": 102100, "latestWkid": 3857}}, "attributes": {"FID": 3, "Address": "2322 Windsor Avenue, Bristol, TN 37620", "Status": "unconnected", "OBJECTID": "65d8d1dae09e67e025e8cdac", "Role": "child"}}


{"geometry": {"x": -9159906.022981752, "y": 4376571.486439344, "spatialReference": {"wkid": 102100, "latestWkid": 3857}}, "attributes": {"FID": 4, "Addre

Network elements with avoid-restrictions are traversed in the output (restriction attribute names: "Avoid Unpaved Roads, Through Traffic Prohibited").
{"cost": 0.44}


I completed the function
               From_OBJECTID                           From_Address From_Role  \
0   65d8d05fe09e67e025e8cda9  220 Fox Glenn Road, Chuckey, TN 37641     child   
1   65d8d05fe09e67e025e8cda9  220 Fox Glenn Road, Chuckey, TN 37641     child   
2   65d8d05fe09e67e025e8cda9  220 Fox Glenn Road, Chuckey, TN 37641     child   
3   65d8d05fe09e67e025e8cda9  220 Fox Glenn Road, Chuckey, TN 37641     child   
4   65d8d05fe09e67e025e8cda9  220 Fox Glenn Road, Chuckey, TN 37641     child   
..                       ...                                    ...       ...   
83  65d8d05fe09e67e025e8cda9  220 Fox Glenn Road, Chuckey, TN 37641     child   
84  65d8d05fe09e67e025e8cda9  220 Fox Glenn Road, Chuckey, TN 37641     child   
85  65d8d05fe09e67e025e8cda9  220 Fox Glenn Road, Chuckey, TN 37641     child   
86  65d8d05fe09e67e025e8cda9  220 Fox Glenn Road, Chuckey, TN 37641     child   
87  65d8d05fe09e67e025e8cda9  220 Fox Glenn Road, Chuckey, TN 37641     child   

  

Network elements with avoid-restrictions are traversed in the output (restriction attribute names: "Avoid Unpaved Roads, Through Traffic Prohibited").
{"cost": 0.435}


I completed the function
               From_OBJECTID                            From_Address  \
0   65d8d0bee09e67e025e8cdaa  18 Glory Court, Jonesborough, TN 37659   
1   65d8d0bee09e67e025e8cdaa  18 Glory Court, Jonesborough, TN 37659   
2   65d8d0bee09e67e025e8cdaa  18 Glory Court, Jonesborough, TN 37659   
3   65d8d0bee09e67e025e8cdaa  18 Glory Court, Jonesborough, TN 37659   
4   65d8d0bee09e67e025e8cdaa  18 Glory Court, Jonesborough, TN 37659   
..                       ...                                     ...   
82  65d8d0bee09e67e025e8cdaa  18 Glory Court, Jonesborough, TN 37659   
83  65d8d0bee09e67e025e8cdaa  18 Glory Court, Jonesborough, TN 37659   
84  65d8d0bee09e67e025e8cdaa  18 Glory Court, Jonesborough, TN 37659   
85  65d8d0bee09e67e025e8cdaa  18 Glory Court, Jonesborough, TN 37659   
86  65d8d0bee09e67e025e8cdaa  18 Glory Court, Jonesborough, TN 37659   

   From_Role               To_OBJECTID  \
0      child  65d8d66de09e67e025e8cdb4   
1      child  65d8d17be09e

...

I completed the function
               From_OBJECTID                            From_Address  \
0   65d8d451e09e67e025e8cdb0  246 Old Stage Trail, Bristol, TN 37620   
1   65d8d451e09e67e025e8cdb0  246 Old Stage Trail, Bristol, TN 37620   
2   65d8d451e09e67e025e8cdb0  246 Old Stage Trail, Bristol, TN 37620   
3   65d8d451e09e67e025e8cdb0  246 Old Stage Trail, Bristol, TN 37620   
4   65d8d451e09e67e025e8cdb0  246 Old Stage Trail, Bristol, TN 37620   
..                       ...                                     ...   
76  65d8d451e09e67e025e8cdb0  246 Old Stage Trail, Bristol, TN 37620   
77  65d8d451e09e67e025e8cdb0  246 Old Stage Trail, Bristol, TN 37620   
78  65d8d451e09e67e025e8cdb0  246 Old Stage Trail, Bristol, TN 37620   
79  65d8d451e09e67e025e8cdb0  246 Old Stage Trail, Bristol, TN 37620   
80  65d8d451e09e67e025e8cdb0  246 Old Stage Trail, Bristol, TN 37620   

   From_Role               To_OBJECTID  \
0      child  65d8d9bce09e67e025e8cdbc   
1      child  65d8d4b3e09e

Network elements with avoid-restrictions are traversed in the output (restriction attribute names: "Avoid Unpaved Roads, Through Traffic Prohibited").
{"cost": 0.4}


I completed the function
               From_OBJECTID                                    From_Address  \
0   65d8d4b3e09e67e025e8cdb1  1218 Pleasant Grove Road, Bluff City, TN 37618   
1   65d8d4b3e09e67e025e8cdb1  1218 Pleasant Grove Road, Bluff City, TN 37618   
2   65d8d4b3e09e67e025e8cdb1  1218 Pleasant Grove Road, Bluff City, TN 37618   
3   65d8d4b3e09e67e025e8cdb1  1218 Pleasant Grove Road, Bluff City, TN 37618   
4   65d8d4b3e09e67e025e8cdb1  1218 Pleasant Grove Road, Bluff City, TN 37618   
..                       ...                                             ...   
75  65d8d4b3e09e67e025e8cdb1  1218 Pleasant Grove Road, Bluff City, TN 37618   
76  65d8d4b3e09e67e025e8cdb1  1218 Pleasant Grove Road, Bluff City, TN 37618   
77  65d8d4b3e09e67e025e8cdb1  1218 Pleasant Grove Road, Bluff City, TN 37618   
78  65d8d4b3e09e67e025e8cdb1  1218 Pleasant Grove Road, Bluff City, TN 37618   
79  65d8d4b3e09e67e025e8cdb1  1218 Pleasant Grove Road, Bluff City, TN 37618   

   From_Role  

Network elements with avoid-restrictions are traversed in the output (restriction attribute names: "Avoid Unpaved Roads, Through Traffic Prohibited").
{"cost": 0.395}


I completed the function
               From_OBJECTID                            From_Address  \
0   65d8d56ae09e67e025e8cdb2  130 Yarrow Drive, Bluff City, TN 37618   
1   65d8d56ae09e67e025e8cdb2  130 Yarrow Drive, Bluff City, TN 37618   
2   65d8d56ae09e67e025e8cdb2  130 Yarrow Drive, Bluff City, TN 37618   
3   65d8d56ae09e67e025e8cdb2  130 Yarrow Drive, Bluff City, TN 37618   
4   65d8d56ae09e67e025e8cdb2  130 Yarrow Drive, Bluff City, TN 37618   
..                       ...                                     ...   
74  65d8d56ae09e67e025e8cdb2  130 Yarrow Drive, Bluff City, TN 37618   
75  65d8d56ae09e67e025e8cdb2  130 Yarrow Drive, Bluff City, TN 37618   
76  65d8d56ae09e67e025e8cdb2  130 Yarrow Drive, Bluff City, TN 37618   
77  65d8d56ae09e67e025e8cdb2  130 Yarrow Drive, Bluff City, TN 37618   
78  65d8d56ae09e67e025e8cdb2  130 Yarrow Drive, Bluff City, TN 37618   

   From_Role               To_OBJECTID  \
0      child  65d8dabde09e67e025e8cdbe   
1      child  65d8d5dbe09e

In [38]:
uniqueDataRemaining = buildData(chunks[11], uniqueDataRemaining, duplicates)

{"geometry": {"x": -9152972.335215973, "y": 4377243.367849248, "spatialReference": {"wkid": 102100, "latestWkid": 3857}}, "attributes": {"FID": 1, "Address": "204 Honeysuckle Lane, Bristol, TN 37620", "Status": "unconnected", "OBJECTID": "65d8d66de09e67e025e8cdb4", "Role": "child"}}


{"geometry": {"x": -9151824.081191067, "y": 4381989.49973961, "spatialReference": {"wkid": 102100, "latestWkid": 3857}}, "attributes": {"FID": 2, "Address": "2608 Bay Street, Bristol, TN 37620", "Status": "unconnected", "OBJECTID": "65d8d6d9e09e67e025e8cdb5", "Role": "child"}}


{"geometry": {"x": -9147593.71632564, "y": 4381479.676133339, "spatialReference": {"wkid": 102100, "latestWkid": 3857}}, "attributes": {"FID": 3, "Address": "822 Tennessee Avenue, Bristol, TN 37620", "Status": "unconnected", "OBJECTID": "65d8d732e09e67e025e8cdb6", "Role": "child"}}


{"geometry": {"x": -9149722.062913224, "y": 4379209.563581477, "spatialReference": {"wkid": 102100, "latestWkid": 3857}}, "attributes": {"FID": 4, "A

Network elements with avoid-restrictions are traversed in the output (restriction attribute names: "Avoid Unpaved Roads, Through Traffic Prohibited").
{"cost": 0.39}


I completed the function
               From_OBJECTID                               From_Address  \
0   65d8d5dbe09e67e025e8cdb3  174 Franklin Drive, Blountville, TN 37617   
1   65d8d5dbe09e67e025e8cdb3  174 Franklin Drive, Blountville, TN 37617   
2   65d8d5dbe09e67e025e8cdb3  174 Franklin Drive, Blountville, TN 37617   
3   65d8d5dbe09e67e025e8cdb3  174 Franklin Drive, Blountville, TN 37617   
4   65d8d5dbe09e67e025e8cdb3  174 Franklin Drive, Blountville, TN 37617   
..                       ...                                        ...   
73  65d8d5dbe09e67e025e8cdb3  174 Franklin Drive, Blountville, TN 37617   
74  65d8d5dbe09e67e025e8cdb3  174 Franklin Drive, Blountville, TN 37617   
75  65d8d5dbe09e67e025e8cdb3  174 Franklin Drive, Blountville, TN 37617   
76  65d8d5dbe09e67e025e8cdb3  174 Franklin Drive, Blountville, TN 37617   
77  65d8d5dbe09e67e025e8cdb3  174 Franklin Drive, Blountville, TN 37617   

   From_Role               To_OBJECTID  \
0      child  65d8db3ce09e67e025

Network elements with avoid-restrictions are traversed in the output (restriction attribute names: "Avoid Unpaved Roads, Through Traffic Prohibited").
{"cost": 0.385}


I completed the function
               From_OBJECTID                             From_Address  \
0   65d8d66de09e67e025e8cdb4  204 Honeysuckle Lane, Bristol, TN 37620   
1   65d8d66de09e67e025e8cdb4  204 Honeysuckle Lane, Bristol, TN 37620   
2   65d8d66de09e67e025e8cdb4  204 Honeysuckle Lane, Bristol, TN 37620   
3   65d8d66de09e67e025e8cdb4  204 Honeysuckle Lane, Bristol, TN 37620   
4   65d8d66de09e67e025e8cdb4  204 Honeysuckle Lane, Bristol, TN 37620   
..                       ...                                      ...   
72  65d8d66de09e67e025e8cdb4  204 Honeysuckle Lane, Bristol, TN 37620   
73  65d8d66de09e67e025e8cdb4  204 Honeysuckle Lane, Bristol, TN 37620   
74  65d8d66de09e67e025e8cdb4  204 Honeysuckle Lane, Bristol, TN 37620   
75  65d8d66de09e67e025e8cdb4  204 Honeysuckle Lane, Bristol, TN 37620   
76  65d8d66de09e67e025e8cdb4  204 Honeysuckle Lane, Bristol, TN 37620   

   From_Role               To_OBJECTID  \
0      child  65d8db8fe09e67e025e8cdc0   
1      child  

...

I completed the function
               From_OBJECTID                         From_Address From_Role  \
0   65d8d9bce09e67e025e8cdbc  121 Anco Drive, Kingsport, TN 37664     child   
1   65d8d9bce09e67e025e8cdbc  121 Anco Drive, Kingsport, TN 37664     child   
2   65d8d9bce09e67e025e8cdbc  121 Anco Drive, Kingsport, TN 37664     child   
3   65d8d9bce09e67e025e8cdbc  121 Anco Drive, Kingsport, TN 37664     child   
4   65d8d9bce09e67e025e8cdbc  121 Anco Drive, Kingsport, TN 37664     child   
..                       ...                                  ...       ...   
66  65d8d9bce09e67e025e8cdbc  121 Anco Drive, Kingsport, TN 37664     child   
67  65d8d9bce09e67e025e8cdbc  121 Anco Drive, Kingsport, TN 37664     child   
68  65d8d9bce09e67e025e8cdbc  121 Anco Drive, Kingsport, TN 37664     child   
69  65d8d9bce09e67e025e8cdbc  121 Anco Drive, Kingsport, TN 37664     child   
70  65d8d9bce09e67e025e8cdbc  121 Anco Drive, Kingsport, TN 37664     child   

                 To_OBJECT

Network elements with avoid-restrictions are traversed in the output (restriction attribute names: "Avoid Unpaved Roads, Through Traffic Prohibited").
{"cost": 0.35000000000000003}


I completed the function
               From_OBJECTID                                  From_Address  \
0   65d8da66e09e67e025e8cdbd  512 East Main Street, Jonesborough, TN 37659   
1   65d8da66e09e67e025e8cdbd  512 East Main Street, Jonesborough, TN 37659   
2   65d8da66e09e67e025e8cdbd  512 East Main Street, Jonesborough, TN 37659   
3   65d8da66e09e67e025e8cdbd  512 East Main Street, Jonesborough, TN 37659   
4   65d8da66e09e67e025e8cdbd  512 East Main Street, Jonesborough, TN 37659   
..                       ...                                           ...   
65  65d8da66e09e67e025e8cdbd  512 East Main Street, Jonesborough, TN 37659   
66  65d8da66e09e67e025e8cdbd  512 East Main Street, Jonesborough, TN 37659   
67  65d8da66e09e67e025e8cdbd  512 East Main Street, Jonesborough, TN 37659   
68  65d8da66e09e67e025e8cdbd  512 East Main Street, Jonesborough, TN 37659   
69  65d8da66e09e67e025e8cdbd  512 East Main Street, Jonesborough, TN 37659   

   From_Role               To_OBJECTID

Network elements with avoid-restrictions are traversed in the output (restriction attribute names: "Avoid Unpaved Roads, Through Traffic Prohibited").
Network elements with avoid-restrictions are traversed in the output (restriction attribute names: "Avoid Gates").
{"cost": 0.34500000000000003}


I completed the function
               From_OBJECTID  \
0   65d8dabde09e67e025e8cdbe   
1   65d8dabde09e67e025e8cdbe   
2   65d8dabde09e67e025e8cdbe   
3   65d8dabde09e67e025e8cdbe   
4   65d8dabde09e67e025e8cdbe   
..                       ...   
64  65d8dabde09e67e025e8cdbe   
65  65d8dabde09e67e025e8cdbe   
66  65d8dabde09e67e025e8cdbe   
67  65d8dabde09e67e025e8cdbe   
68  65d8dabde09e67e025e8cdbe   

                                         From_Address From_Role  \
0   338 South Broadway Street, Johnson City, TN 37601     child   
1   338 South Broadway Street, Johnson City, TN 37601     child   
2   338 South Broadway Street, Johnson City, TN 37601     child   
3   338 South Broadway Street, Johnson City, TN 37601     child   
4   338 South Broadway Street, Johnson City, TN 37601     child   
..                                                ...       ...   
64  338 South Broadway Street, Johnson City, TN 37601     child   
65  338 South Broadway Street, Johnson City, TN 37601 

In [39]:
uniqueDataRemaining = buildData(chunks[12], uniqueDataRemaining, duplicates)

{"geometry": {"x": -9118500.934059987, "y": 4371270.334779087, "spatialReference": {"wkid": 102100, "latestWkid": 3857}}, "attributes": {"FID": 1, "Address": "180 Martin Lane, Shady Valley, TN 37688", "Status": "unconnected", "OBJECTID": "65d8db8fe09e67e025e8cdc0", "Role": "child"}}


{"geometry": {"x": -9147206.435382327, "y": 4340064.04276909, "spatialReference": {"wkid": 102100, "latestWkid": 3857}}, "attributes": {"FID": 2, "Address": "105 Second Avenue, Hampton, TN 37658", "Status": "unconnected", "OBJECTID": "65e5c8f938f826f292671fe9", "Role": "child"}}


{"geometry": {"x": -9171648.449126985, "y": 4342373.31792813, "spatialReference": {"wkid": 102100, "latestWkid": 3857}}, "attributes": {"FID": 3, "Address": "2370 Textile St, Johnson City, TN 37604", "Status": "unconnected", "OBJECTID": "65e5ca5838f826f292671fea", "Role": "child"}}


{"geometry": {"x": -9131727.087854266, "y": 4348970.888386844, "spatialReference": {"wkid": 102100, "latestWkid": 3857}}, "attributes": {"FID": 4, 

Network elements with avoid-restrictions are traversed in the output (restriction attribute names: "Avoid Unpaved Roads, Through Traffic Prohibited").
{"cost": 0.34}


I completed the function
               From_OBJECTID                                From_Address  \
0   65d8db3ce09e67e025e8cdbf  160 Sycamore Drive, Jonesborough, TN 37659   
1   65d8db3ce09e67e025e8cdbf  160 Sycamore Drive, Jonesborough, TN 37659   
2   65d8db3ce09e67e025e8cdbf  160 Sycamore Drive, Jonesborough, TN 37659   
3   65d8db3ce09e67e025e8cdbf  160 Sycamore Drive, Jonesborough, TN 37659   
4   65d8db3ce09e67e025e8cdbf  160 Sycamore Drive, Jonesborough, TN 37659   
..                       ...                                         ...   
63  65d8db3ce09e67e025e8cdbf  160 Sycamore Drive, Jonesborough, TN 37659   
64  65d8db3ce09e67e025e8cdbf  160 Sycamore Drive, Jonesborough, TN 37659   
65  65d8db3ce09e67e025e8cdbf  160 Sycamore Drive, Jonesborough, TN 37659   
66  65d8db3ce09e67e025e8cdbf  160 Sycamore Drive, Jonesborough, TN 37659   
67  65d8db3ce09e67e025e8cdbf  160 Sycamore Drive, Jonesborough, TN 37659   

   From_Role               To_OBJECTID  \
0      child  65e5ce

Network elements with avoid-restrictions are traversed in the output (restriction attribute names: "Avoid Unpaved Roads, Through Traffic Prohibited").
Network elements with avoid-restrictions are traversed in the output (restriction attribute names: "Avoid Gates").
{"cost": 0.335}


I completed the function
               From_OBJECTID                             From_Address  \
0   65d8db8fe09e67e025e8cdc0  180 Martin Lane, Shady Valley, TN 37688   
1   65d8db8fe09e67e025e8cdc0  180 Martin Lane, Shady Valley, TN 37688   
2   65d8db8fe09e67e025e8cdc0  180 Martin Lane, Shady Valley, TN 37688   
3   65d8db8fe09e67e025e8cdc0  180 Martin Lane, Shady Valley, TN 37688   
4   65d8db8fe09e67e025e8cdc0  180 Martin Lane, Shady Valley, TN 37688   
..                       ...                                      ...   
62  65d8db8fe09e67e025e8cdc0  180 Martin Lane, Shady Valley, TN 37688   
63  65d8db8fe09e67e025e8cdc0  180 Martin Lane, Shady Valley, TN 37688   
64  65d8db8fe09e67e025e8cdc0  180 Martin Lane, Shady Valley, TN 37688   
65  65d8db8fe09e67e025e8cdc0  180 Martin Lane, Shady Valley, TN 37688   
66  65d8db8fe09e67e025e8cdc0  180 Martin Lane, Shady Valley, TN 37688   

   From_Role               To_OBJECTID  \
0      child  65e5cfc338f826f292671ff3   
1      child  

...

I completed the function
               From_OBJECTID                              From_Address  \
0   65e5cd3c38f826f292671fef  512 Swadley Road, Johnson City, TN 37601   
1   65e5cd3c38f826f292671fef  512 Swadley Road, Johnson City, TN 37601   
2   65e5cd3c38f826f292671fef  512 Swadley Road, Johnson City, TN 37601   
3   65e5cd3c38f826f292671fef  512 Swadley Road, Johnson City, TN 37601   
4   65e5cd3c38f826f292671fef  512 Swadley Road, Johnson City, TN 37601   
..                       ...                                       ...   
56  65e5cd3c38f826f292671fef  512 Swadley Road, Johnson City, TN 37601   
57  65e5cd3c38f826f292671fef  512 Swadley Road, Johnson City, TN 37601   
58  65e5cd3c38f826f292671fef  512 Swadley Road, Johnson City, TN 37601   
59  65e5cd3c38f826f292671fef  512 Swadley Road, Johnson City, TN 37601   
60  65e5cd3c38f826f292671fef  512 Swadley Road, Johnson City, TN 37601   

   From_Role               To_OBJECTID  \
0      child  65e5d38f38f826f292671ffa   
1 

Network elements with avoid-restrictions are traversed in the output (restriction attribute names: "Avoid Unpaved Roads, Through Traffic Prohibited").
{"cost": 0.3}


I completed the function
               From_OBJECTID                               From_Address  \
0   65e5cdba38f826f292671ff0  1508 Squibb Drive, Johnson City, TN 37601   
1   65e5cdba38f826f292671ff0  1508 Squibb Drive, Johnson City, TN 37601   
2   65e5cdba38f826f292671ff0  1508 Squibb Drive, Johnson City, TN 37601   
3   65e5cdba38f826f292671ff0  1508 Squibb Drive, Johnson City, TN 37601   
4   65e5cdba38f826f292671ff0  1508 Squibb Drive, Johnson City, TN 37601   
5   65e5cdba38f826f292671ff0  1508 Squibb Drive, Johnson City, TN 37601   
6   65e5cdba38f826f292671ff0  1508 Squibb Drive, Johnson City, TN 37601   
7   65e5cdba38f826f292671ff0  1508 Squibb Drive, Johnson City, TN 37601   
8   65e5cdba38f826f292671ff0  1508 Squibb Drive, Johnson City, TN 37601   
9   65e5cdba38f826f292671ff0  1508 Squibb Drive, Johnson City, TN 37601   
10  65e5cdba38f826f292671ff0  1508 Squibb Drive, Johnson City, TN 37601   
11  65e5cdba38f826f292671ff0  1508 Squibb Drive, Johnson City, TN 37601   


Network elements with avoid-restrictions are traversed in the output (restriction attribute names: "Avoid Unpaved Roads, Through Traffic Prohibited").
{"cost": 0.295}


I completed the function
               From_OBJECTID                       From_Address From_Role  \
0   65e5ce6238f826f292671ff1  105 Canaan Road, Butler, TN 37640     child   
1   65e5ce6238f826f292671ff1  105 Canaan Road, Butler, TN 37640     child   
2   65e5ce6238f826f292671ff1  105 Canaan Road, Butler, TN 37640     child   
3   65e5ce6238f826f292671ff1  105 Canaan Road, Butler, TN 37640     child   
4   65e5ce6238f826f292671ff1  105 Canaan Road, Butler, TN 37640     child   
5   65e5ce6238f826f292671ff1  105 Canaan Road, Butler, TN 37640     child   
6   65e5ce6238f826f292671ff1  105 Canaan Road, Butler, TN 37640     child   
7   65e5ce6238f826f292671ff1  105 Canaan Road, Butler, TN 37640     child   
8   65e5ce6238f826f292671ff1  105 Canaan Road, Butler, TN 37640     child   
9   65e5ce6238f826f292671ff1  105 Canaan Road, Butler, TN 37640     child   
10  65e5ce6238f826f292671ff1  105 Canaan Road, Butler, TN 37640     child   
11  65e5ce6238f826f292671ff1  105 Canaan Road, Butl

In [40]:
uniqueDataRemaining = buildData(chunks[13], uniqueDataRemaining, duplicates)

{"geometry": {"x": -9153424.16298317, "y": 4328532.447014573, "spatialReference": {"wkid": 102100, "latestWkid": 3857}}, "attributes": {"FID": 1, "Address": "922 Fairview Road, Hampton, TN 37658", "Status": "unconnected", "OBJECTID": "65e5cfc338f826f292671ff3", "Role": "child"}}


{"geometry": {"x": -9142234.631342534, "y": 4354875.603616882, "spatialReference": {"wkid": 102100, "latestWkid": 3857}}, "attributes": {"FID": 2, "Address": "202 Spindale Road, Elizabethton, TN 37643", "Status": "unconnected", "OBJECTID": "65e5d02938f826f292671ff4", "Role": "child"}}


{"geometry": {"x": -9141361.550619453, "y": 4319764.443792437, "spatialReference": {"wkid": 102100, "latestWkid": 3857}}, "attributes": {"FID": 3, "Address": "104 Buck Road, Roan Mountain, TN 37687", "Status": "unconnected", "OBJECTID": "65e5d0cf38f826f292671ff5", "Role": "child"}}


{"geometry": {"x": -9143177.354691384, "y": 4380524.615535399, "spatialReference": {"wkid": 102100, "latestWkid": 3857}}, "attributes": {"FID": 4

Network elements with avoid-restrictions are traversed in the output (restriction attribute names: "Avoid Unpaved Roads, Through Traffic Prohibited").
{"cost": 0.29}


I completed the function
               From_OBJECTID                                    From_Address  \
0   65e5cef438f826f292671ff2  300 White Thorn Trail, Roan Mountain, TN 37687   
1   65e5cef438f826f292671ff2  300 White Thorn Trail, Roan Mountain, TN 37687   
2   65e5cef438f826f292671ff2  300 White Thorn Trail, Roan Mountain, TN 37687   
3   65e5cef438f826f292671ff2  300 White Thorn Trail, Roan Mountain, TN 37687   
4   65e5cef438f826f292671ff2  300 White Thorn Trail, Roan Mountain, TN 37687   
5   65e5cef438f826f292671ff2  300 White Thorn Trail, Roan Mountain, TN 37687   
6   65e5cef438f826f292671ff2  300 White Thorn Trail, Roan Mountain, TN 37687   
7   65e5cef438f826f292671ff2  300 White Thorn Trail, Roan Mountain, TN 37687   
8   65e5cef438f826f292671ff2  300 White Thorn Trail, Roan Mountain, TN 37687   
9   65e5cef438f826f292671ff2  300 White Thorn Trail, Roan Mountain, TN 37687   
10  65e5cef438f826f292671ff2  300 White Thorn Trail, Roan Mountain, TN 37687   
11  65e5cef438f

Network elements with avoid-restrictions are traversed in the output (restriction attribute names: "Avoid Unpaved Roads, Through Traffic Prohibited").
{"cost": 0.28500000000000003}


I completed the function
               From_OBJECTID                          From_Address From_Role  \
0   65e5cfc338f826f292671ff3  922 Fairview Road, Hampton, TN 37658     child   
1   65e5cfc338f826f292671ff3  922 Fairview Road, Hampton, TN 37658     child   
2   65e5cfc338f826f292671ff3  922 Fairview Road, Hampton, TN 37658     child   
3   65e5cfc338f826f292671ff3  922 Fairview Road, Hampton, TN 37658     child   
4   65e5cfc338f826f292671ff3  922 Fairview Road, Hampton, TN 37658     child   
5   65e5cfc338f826f292671ff3  922 Fairview Road, Hampton, TN 37658     child   
6   65e5cfc338f826f292671ff3  922 Fairview Road, Hampton, TN 37658     child   
7   65e5cfc338f826f292671ff3  922 Fairview Road, Hampton, TN 37658     child   
8   65e5cfc338f826f292671ff3  922 Fairview Road, Hampton, TN 37658     child   
9   65e5cfc338f826f292671ff3  922 Fairview Road, Hampton, TN 37658     child   
10  65e5cfc338f826f292671ff3  922 Fairview Road, Hampton, TN 37658     child   
11  65e5cfc338f

...

I completed the function
               From_OBJECTID                                    From_Address  \
0   65e5d38f38f826f292671ffa  173 Ollie Collins Road, Elizabethton, TN 37643   
1   65e5d38f38f826f292671ffa  173 Ollie Collins Road, Elizabethton, TN 37643   
2   65e5d38f38f826f292671ffa  173 Ollie Collins Road, Elizabethton, TN 37643   
3   65e5d38f38f826f292671ffa  173 Ollie Collins Road, Elizabethton, TN 37643   
4   65e5d38f38f826f292671ffa  173 Ollie Collins Road, Elizabethton, TN 37643   
5   65e5d38f38f826f292671ffa  173 Ollie Collins Road, Elizabethton, TN 37643   
6   65e5d38f38f826f292671ffa  173 Ollie Collins Road, Elizabethton, TN 37643   
7   65e5d38f38f826f292671ffa  173 Ollie Collins Road, Elizabethton, TN 37643   
8   65e5d38f38f826f292671ffa  173 Ollie Collins Road, Elizabethton, TN 37643   
9   65e5d38f38f826f292671ffa  173 Ollie Collins Road, Elizabethton, TN 37643   
10  65e5d38f38f826f292671ffa  173 Ollie Collins Road, Elizabethton, TN 37643   
11  65e5d38f38f

Network elements with avoid-restrictions are traversed in the output (restriction attribute names: "Avoid Unpaved Roads, Through Traffic Prohibited").
{"cost": 0.25}


I completed the function
               From_OBJECTID                              From_Address  \
0   65e5d41e38f826f292671ffb  209 Hollydale Drive, Kingsport, TN 37663   
1   65e5d41e38f826f292671ffb  209 Hollydale Drive, Kingsport, TN 37663   
2   65e5d41e38f826f292671ffb  209 Hollydale Drive, Kingsport, TN 37663   
3   65e5d41e38f826f292671ffb  209 Hollydale Drive, Kingsport, TN 37663   
4   65e5d41e38f826f292671ffb  209 Hollydale Drive, Kingsport, TN 37663   
5   65e5d41e38f826f292671ffb  209 Hollydale Drive, Kingsport, TN 37663   
6   65e5d41e38f826f292671ffb  209 Hollydale Drive, Kingsport, TN 37663   
7   65e5d41e38f826f292671ffb  209 Hollydale Drive, Kingsport, TN 37663   
8   65e5d41e38f826f292671ffb  209 Hollydale Drive, Kingsport, TN 37663   
9   65e5d41e38f826f292671ffb  209 Hollydale Drive, Kingsport, TN 37663   
10  65e5d41e38f826f292671ffb  209 Hollydale Drive, Kingsport, TN 37663   
11  65e5d41e38f826f292671ffb  209 Hollydale Drive, Kingsport, TN 37663   
12  65e5d41e3

Network elements with avoid-restrictions are traversed in the output (restriction attribute names: "Avoid Unpaved Roads, Through Traffic Prohibited").
{"cost": 0.245}


I completed the function
               From_OBJECTID                                  From_Address  \
0   65e5d4d238f826f292671ffc  6345 Wildlife Circle, Piney Flats, TN 376086   
1   65e5d4d238f826f292671ffc  6345 Wildlife Circle, Piney Flats, TN 376086   
2   65e5d4d238f826f292671ffc  6345 Wildlife Circle, Piney Flats, TN 376086   
3   65e5d4d238f826f292671ffc  6345 Wildlife Circle, Piney Flats, TN 376086   
4   65e5d4d238f826f292671ffc  6345 Wildlife Circle, Piney Flats, TN 376086   
5   65e5d4d238f826f292671ffc  6345 Wildlife Circle, Piney Flats, TN 376086   
6   65e5d4d238f826f292671ffc  6345 Wildlife Circle, Piney Flats, TN 376086   
7   65e5d4d238f826f292671ffc  6345 Wildlife Circle, Piney Flats, TN 376086   
8   65e5d4d238f826f292671ffc  6345 Wildlife Circle, Piney Flats, TN 376086   
9   65e5d4d238f826f292671ffc  6345 Wildlife Circle, Piney Flats, TN 376086   
10  65e5d4d238f826f292671ffc  6345 Wildlife Circle, Piney Flats, TN 376086   
11  65e5d4d238f826f292671ffc  6345 Wild

In [41]:
uniqueDataRemaining = buildData(chunks[14], uniqueDataRemaining, duplicates)

{"geometry": {"x": -9157826.086512081, "y": 4365378.251969403, "spatialReference": {"wkid": 102100, "latestWkid": 3857}}, "attributes": {"FID": 1, "Address": "309 Fox Private Drive, Bluff City, TN 37618", "Status": "unconnected", "OBJECTID": "65e5d62e38f826f292671fff", "Role": "child"}}


{"geometry": {"x": -9152670.833604403, "y": 4346429.841695883, "spatialReference": {"wkid": 102100, "latestWkid": 3857}}, "attributes": {"FID": 2, "Address": "307 South View Circle, Elizabethton, TN 37643", "Status": "unconnected", "OBJECTID": "65e5d6b638f826f292672000", "Role": "child"}}


{"geometry": {"x": -9169778.28983494, "y": 4357180.1449166825, "spatialReference": {"wkid": 102100, "latestWkid": 3857}}, "attributes": {"FID": 3, "Address": "2336 Egret Lane, Piney Flats, TN 37686", "Status": "unconnected", "OBJECTID": "65e5d71c38f826f292672001", "Role": "child"}}


{"geometry": {"x": -9172135.94750738, "y": 4367322.730424997, "spatialReference": {"wkid": 102100, "latestWkid": 3857}}, "attributes"

Network elements with avoid-restrictions are traversed in the output (restriction attribute names: "Avoid Unpaved Roads, Through Traffic Prohibited").
{"cost": 0.24}


I completed the function
               From_OBJECTID                                   From_Address  \
0   65e5d5d038f826f292671ffe  925 Pine Ridge Circle, Elizabethton, TN 37643   
1   65e5d5d038f826f292671ffe  925 Pine Ridge Circle, Elizabethton, TN 37643   
2   65e5d5d038f826f292671ffe  925 Pine Ridge Circle, Elizabethton, TN 37643   
3   65e5d5d038f826f292671ffe  925 Pine Ridge Circle, Elizabethton, TN 37643   
4   65e5d5d038f826f292671ffe  925 Pine Ridge Circle, Elizabethton, TN 37643   
5   65e5d5d038f826f292671ffe  925 Pine Ridge Circle, Elizabethton, TN 37643   
6   65e5d5d038f826f292671ffe  925 Pine Ridge Circle, Elizabethton, TN 37643   
7   65e5d5d038f826f292671ffe  925 Pine Ridge Circle, Elizabethton, TN 37643   
8   65e5d5d038f826f292671ffe  925 Pine Ridge Circle, Elizabethton, TN 37643   
9   65e5d5d038f826f292671ffe  925 Pine Ridge Circle, Elizabethton, TN 37643   
10  65e5d5d038f826f292671ffe  925 Pine Ridge Circle, Elizabethton, TN 37643   
11  65e5d5d038f826f292671ff

Network elements with avoid-restrictions are traversed in the output (restriction attribute names: "Avoid Unpaved Roads, Through Traffic Prohibited").
{"cost": 0.23500000000000001}


I completed the function
               From_OBJECTID                                 From_Address  \
0   65e5d62e38f826f292671fff  309 Fox Private Drive, Bluff City, TN 37618   
1   65e5d62e38f826f292671fff  309 Fox Private Drive, Bluff City, TN 37618   
2   65e5d62e38f826f292671fff  309 Fox Private Drive, Bluff City, TN 37618   
3   65e5d62e38f826f292671fff  309 Fox Private Drive, Bluff City, TN 37618   
4   65e5d62e38f826f292671fff  309 Fox Private Drive, Bluff City, TN 37618   
5   65e5d62e38f826f292671fff  309 Fox Private Drive, Bluff City, TN 37618   
6   65e5d62e38f826f292671fff  309 Fox Private Drive, Bluff City, TN 37618   
7   65e5d62e38f826f292671fff  309 Fox Private Drive, Bluff City, TN 37618   
8   65e5d62e38f826f292671fff  309 Fox Private Drive, Bluff City, TN 37618   
9   65e5d62e38f826f292671fff  309 Fox Private Drive, Bluff City, TN 37618   
10  65e5d62e38f826f292671fff  309 Fox Private Drive, Bluff City, TN 37618   
11  65e5d62e38f826f292671fff  309 Fox Private Drive

...

I completed the function
               From_OBJECTID                                 From_Address  \
0   65e5d95538f826f292672006  1618 Hillview Drive, Elizabethton, TN 37643   
1   65e5d95538f826f292672006  1618 Hillview Drive, Elizabethton, TN 37643   
2   65e5d95538f826f292672006  1618 Hillview Drive, Elizabethton, TN 37643   
3   65e5d95538f826f292672006  1618 Hillview Drive, Elizabethton, TN 37643   
4   65e5d95538f826f292672006  1618 Hillview Drive, Elizabethton, TN 37643   
5   65e5d95538f826f292672006  1618 Hillview Drive, Elizabethton, TN 37643   
6   65e5d95538f826f292672006  1618 Hillview Drive, Elizabethton, TN 37643   
7   65e5d95538f826f292672006  1618 Hillview Drive, Elizabethton, TN 37643   
8   65e5d95538f826f292672006  1618 Hillview Drive, Elizabethton, TN 37643   
9   65e5d95538f826f292672006  1618 Hillview Drive, Elizabethton, TN 37643   
10  65e5d95538f826f292672006  1618 Hillview Drive, Elizabethton, TN 37643   
11  65e5d95538f826f292672006  1618 Hillview Drive, 

{"cost": 0.2}


I completed the function
               From_OBJECTID                                    From_Address  \
0   65e5d9ad38f826f292672007  236 Claude Brown Lane, Mountain City, TN 37683   
1   65e5d9ad38f826f292672007  236 Claude Brown Lane, Mountain City, TN 37683   
2   65e5d9ad38f826f292672007  236 Claude Brown Lane, Mountain City, TN 37683   
3   65e5d9ad38f826f292672007  236 Claude Brown Lane, Mountain City, TN 37683   
4   65e5d9ad38f826f292672007  236 Claude Brown Lane, Mountain City, TN 37683   
5   65e5d9ad38f826f292672007  236 Claude Brown Lane, Mountain City, TN 37683   
6   65e5d9ad38f826f292672007  236 Claude Brown Lane, Mountain City, TN 37683   
7   65e5d9ad38f826f292672007  236 Claude Brown Lane, Mountain City, TN 37683   
8   65e5d9ad38f826f292672007  236 Claude Brown Lane, Mountain City, TN 37683   
9   65e5d9ad38f826f292672007  236 Claude Brown Lane, Mountain City, TN 37683   
10  65e5d9ad38f826f292672007  236 Claude Brown Lane, Mountain City, TN 37683   
11  65e5d9ad38f

{"cost": 0.195}


I completed the function
               From_OBJECTID                               From_Address  \
0   65e5da5138f826f292672008  246 Taylor Avenue, Elizabethton, TN 37643   
1   65e5da5138f826f292672008  246 Taylor Avenue, Elizabethton, TN 37643   
2   65e5da5138f826f292672008  246 Taylor Avenue, Elizabethton, TN 37643   
3   65e5da5138f826f292672008  246 Taylor Avenue, Elizabethton, TN 37643   
4   65e5da5138f826f292672008  246 Taylor Avenue, Elizabethton, TN 37643   
5   65e5da5138f826f292672008  246 Taylor Avenue, Elizabethton, TN 37643   
6   65e5da5138f826f292672008  246 Taylor Avenue, Elizabethton, TN 37643   
7   65e5da5138f826f292672008  246 Taylor Avenue, Elizabethton, TN 37643   
8   65e5da5138f826f292672008  246 Taylor Avenue, Elizabethton, TN 37643   
9   65e5da5138f826f292672008  246 Taylor Avenue, Elizabethton, TN 37643   
10  65e5da5138f826f292672008  246 Taylor Avenue, Elizabethton, TN 37643   
11  65e5da5138f826f292672008  246 Taylor Avenue, Elizabethton, TN 37643   


In [42]:
uniqueDataRemaining = buildData(chunks[15], uniqueDataRemaining, duplicates)

{"geometry": {"x": -9164867.635417774, "y": 4342377.5333478935, "spatialReference": {"wkid": 102100, "latestWkid": 3857}}, "attributes": {"FID": 1, "Address": "2398 Lakewood Drive, Johnson City, TN 37601", "Status": "unconnected", "OBJECTID": "65e5db4d38f826f29267200a", "Role": "child"}}


{"geometry": {"x": -9176804.551334975, "y": 4343302.330935071, "spatialReference": {"wkid": 102100, "latestWkid": 3857}}, "attributes": {"FID": 2, "Address": "3783 West Market Street, Johnson City, TN 37604", "Status": "unconnected", "OBJECTID": "65e5db9f38f826f29267200b", "Role": "child"}}


{"geometry": {"x": -9177692.919463713, "y": 4357486.1880283905, "spatialReference": {"wkid": 102100, "latestWkid": 3857}}, "attributes": {"FID": 3, "Address": "1120 Cliffview Circle, Johnson City, TN 37615", "Status": "unconnected", "OBJECTID": "65e5dbff38f826f29267200c", "Role": "child"}}


{"geometry": {"x": -9176655.074479664, "y": 4359523.179800584, "spatialReference": {"wkid": 102100, "latestWkid": 3857}}, 

{"cost": 0.19}


I completed the function
               From_OBJECTID  \
0   65e5daec38f826f292672009   
1   65e5daec38f826f292672009   
2   65e5daec38f826f292672009   
3   65e5daec38f826f292672009   
4   65e5daec38f826f292672009   
5   65e5daec38f826f292672009   
6   65e5daec38f826f292672009   
7   65e5daec38f826f292672009   
8   65e5daec38f826f292672009   
9   65e5daec38f826f292672009   
10  65e5daec38f826f292672009   
11  65e5daec38f826f292672009   
12  65e5daec38f826f292672009   
13  65e5daec38f826f292672009   
14  65e5daec38f826f292672009   
15  65e5daec38f826f292672009   
16  65e5daec38f826f292672009   
17  65e5daec38f826f292672009   
18  65e5daec38f826f292672009   
19  65e5daec38f826f292672009   
20  65e5daec38f826f292672009   
21  65e5daec38f826f292672009   
22  65e5daec38f826f292672009   
23  65e5daec38f826f292672009   
24  65e5daec38f826f292672009   
25  65e5daec38f826f292672009   
26  65e5daec38f826f292672009   
27  65e5daec38f826f292672009   
28  65e5daec38f826f292672009   
29  65e5daec38f

{"cost": 0.185}


I completed the function
               From_OBJECTID                                 From_Address  \
0   65e5db4d38f826f29267200a  2398 Lakewood Drive, Johnson City, TN 37601   
1   65e5db4d38f826f29267200a  2398 Lakewood Drive, Johnson City, TN 37601   
2   65e5db4d38f826f29267200a  2398 Lakewood Drive, Johnson City, TN 37601   
3   65e5db4d38f826f29267200a  2398 Lakewood Drive, Johnson City, TN 37601   
4   65e5db4d38f826f29267200a  2398 Lakewood Drive, Johnson City, TN 37601   
5   65e5db4d38f826f29267200a  2398 Lakewood Drive, Johnson City, TN 37601   
6   65e5db4d38f826f29267200a  2398 Lakewood Drive, Johnson City, TN 37601   
7   65e5db4d38f826f29267200a  2398 Lakewood Drive, Johnson City, TN 37601   
8   65e5db4d38f826f29267200a  2398 Lakewood Drive, Johnson City, TN 37601   
9   65e5db4d38f826f29267200a  2398 Lakewood Drive, Johnson City, TN 37601   
10  65e5db4d38f826f29267200a  2398 Lakewood Drive, Johnson City, TN 37601   
11  65e5db4d38f826f29267200a  2398 Lakewood Drive, 

...

I completed the function
               From_OBJECTID                                 From_Address  \
0   65e5dd8a38f826f292672010  200 Colorado Street, Johnson City, TN 37601   
1   65e5dd8a38f826f292672010  200 Colorado Street, Johnson City, TN 37601   
2   65e5dd8a38f826f292672010  200 Colorado Street, Johnson City, TN 37601   
3   65e5dd8a38f826f292672010  200 Colorado Street, Johnson City, TN 37601   
4   65e5dd8a38f826f292672010  200 Colorado Street, Johnson City, TN 37601   
5   65e5dd8a38f826f292672010  200 Colorado Street, Johnson City, TN 37601   
6   65e5dd8a38f826f292672010  200 Colorado Street, Johnson City, TN 37601   
7   65e5dd8a38f826f292672010  200 Colorado Street, Johnson City, TN 37601   
8   65e5dd8a38f826f292672010  200 Colorado Street, Johnson City, TN 37601   
9   65e5dd8a38f826f292672010  200 Colorado Street, Johnson City, TN 37601   
10  65e5dd8a38f826f292672010  200 Colorado Street, Johnson City, TN 37601   
11  65e5dd8a38f826f292672010  200 Colorado Street, 

{"cost": 0.15}


I completed the function
               From_OBJECTID                                From_Address  \
0   65e5dded38f826f292672011  917 Bunker Hill Road, Bluff City, TN 37618   
1   65e5dded38f826f292672011  917 Bunker Hill Road, Bluff City, TN 37618   
2   65e5dded38f826f292672011  917 Bunker Hill Road, Bluff City, TN 37618   
3   65e5dded38f826f292672011  917 Bunker Hill Road, Bluff City, TN 37618   
4   65e5dded38f826f292672011  917 Bunker Hill Road, Bluff City, TN 37618   
5   65e5dded38f826f292672011  917 Bunker Hill Road, Bluff City, TN 37618   
6   65e5dded38f826f292672011  917 Bunker Hill Road, Bluff City, TN 37618   
7   65e5dded38f826f292672011  917 Bunker Hill Road, Bluff City, TN 37618   
8   65e5dded38f826f292672011  917 Bunker Hill Road, Bluff City, TN 37618   
9   65e5dded38f826f292672011  917 Bunker Hill Road, Bluff City, TN 37618   
10  65e5dded38f826f292672011  917 Bunker Hill Road, Bluff City, TN 37618   
11  65e5dded38f826f292672011  917 Bunker Hill Road, Bluff City,

{"cost": 0.145}


I completed the function
               From_OBJECTID                                   From_Address  \
0   65e5dec738f826f292672012  513 Washington Avenue, Johnson City, TN 37604   
1   65e5dec738f826f292672012  513 Washington Avenue, Johnson City, TN 37604   
2   65e5dec738f826f292672012  513 Washington Avenue, Johnson City, TN 37604   
3   65e5dec738f826f292672012  513 Washington Avenue, Johnson City, TN 37604   
4   65e5dec738f826f292672012  513 Washington Avenue, Johnson City, TN 37604   
5   65e5dec738f826f292672012  513 Washington Avenue, Johnson City, TN 37604   
6   65e5dec738f826f292672012  513 Washington Avenue, Johnson City, TN 37604   
7   65e5dec738f826f292672012  513 Washington Avenue, Johnson City, TN 37604   
8   65e5dec738f826f292672012  513 Washington Avenue, Johnson City, TN 37604   
9   65e5dec738f826f292672012  513 Washington Avenue, Johnson City, TN 37604   
10  65e5dec738f826f292672012  513 Washington Avenue, Johnson City, TN 37604   
11  65e5dec738f826f29267201

In [43]:
uniqueDataRemaining = buildData(chunks[16], uniqueDataRemaining, duplicates)

{"geometry": {"x": -9165647.293106282, "y": 4348047.185415673, "spatialReference": {"wkid": 102100, "latestWkid": 3857}}, "attributes": {"FID": 1, "Address": "918 Pine Grove Avenue, Johnson City, TN 37601", "Status": "unconnected", "OBJECTID": "65e5df8638f826f292672014", "Role": "child"}}


{"geometry": {"x": -9176785.866728062, "y": 4359547.457161156, "spatialReference": {"wkid": 102100, "latestWkid": 3857}}, "attributes": {"FID": 2, "Address": "111 Hicks Drive, Gray, TN 37615", "Status": "unconnected", "OBJECTID": "65e5e03138f826f292672016", "Role": "child"}}


{"geometry": {"x": -9168691.185432663, "y": 4350312.824208585, "spatialReference": {"wkid": 102100, "latestWkid": 3857}}, "attributes": {"FID": 3, "Address": "658 Princeton Road, Johnson City, TN 37601", "Status": "unconnected", "OBJECTID": "65e5e0b738f826f292672017", "Role": "child"}}


{"geometry": {"x": -9170253.048710622, "y": 4341219.935661357, "spatialReference": {"wkid": 102100, "latestWkid": 3857}}, "attributes": {"FID

{"cost": 0.14}


I completed the function
               From_OBJECTID                                    From_Address  \
0   65e5df2638f826f292672013  2717 South Roan Street, Johnson City, TN 37601   
1   65e5df2638f826f292672013  2717 South Roan Street, Johnson City, TN 37601   
2   65e5df2638f826f292672013  2717 South Roan Street, Johnson City, TN 37601   
3   65e5df2638f826f292672013  2717 South Roan Street, Johnson City, TN 37601   
4   65e5df2638f826f292672013  2717 South Roan Street, Johnson City, TN 37601   
5   65e5df2638f826f292672013  2717 South Roan Street, Johnson City, TN 37601   
6   65e5df2638f826f292672013  2717 South Roan Street, Johnson City, TN 37601   
7   65e5df2638f826f292672013  2717 South Roan Street, Johnson City, TN 37601   
8   65e5df2638f826f292672013  2717 South Roan Street, Johnson City, TN 37601   
9   65e5df2638f826f292672013  2717 South Roan Street, Johnson City, TN 37601   
10  65e5df2638f826f292672013  2717 South Roan Street, Johnson City, TN 37601   
11  65e5df2638f

{"cost": 0.135}


I completed the function
               From_OBJECTID                                   From_Address  \
0   65e5df8638f826f292672014  918 Pine Grove Avenue, Johnson City, TN 37601   
1   65e5df8638f826f292672014  918 Pine Grove Avenue, Johnson City, TN 37601   
2   65e5df8638f826f292672014  918 Pine Grove Avenue, Johnson City, TN 37601   
3   65e5df8638f826f292672014  918 Pine Grove Avenue, Johnson City, TN 37601   
4   65e5df8638f826f292672014  918 Pine Grove Avenue, Johnson City, TN 37601   
5   65e5df8638f826f292672014  918 Pine Grove Avenue, Johnson City, TN 37601   
6   65e5df8638f826f292672014  918 Pine Grove Avenue, Johnson City, TN 37601   
7   65e5df8638f826f292672014  918 Pine Grove Avenue, Johnson City, TN 37601   
8   65e5df8638f826f292672014  918 Pine Grove Avenue, Johnson City, TN 37601   
9   65e5df8638f826f292672014  918 Pine Grove Avenue, Johnson City, TN 37601   
10  65e5df8638f826f292672014  918 Pine Grove Avenue, Johnson City, TN 37601   
11  65e5df8638f826f29267201

...

I completed the function
               From_OBJECTID                             From_Address  \
0   65e5e2dc38f826f29267201b  902 Ashely Road, Johnson City, TN 37604   
1   65e5e2dc38f826f29267201b  902 Ashely Road, Johnson City, TN 37604   
2   65e5e2dc38f826f29267201b  902 Ashely Road, Johnson City, TN 37604   
3   65e5e2dc38f826f29267201b  902 Ashely Road, Johnson City, TN 37604   
4   65e5e2dc38f826f29267201b  902 Ashely Road, Johnson City, TN 37604   
5   65e5e2dc38f826f29267201b  902 Ashely Road, Johnson City, TN 37604   
6   65e5e2dc38f826f29267201b  902 Ashely Road, Johnson City, TN 37604   
7   65e5e2dc38f826f29267201b  902 Ashely Road, Johnson City, TN 37604   
8   65e5e2dc38f826f29267201b  902 Ashely Road, Johnson City, TN 37604   
9   65e5e2dc38f826f29267201b  902 Ashely Road, Johnson City, TN 37604   
10  65e5e2dc38f826f29267201b  902 Ashely Road, Johnson City, TN 37604   
11  65e5e2dc38f826f29267201b  902 Ashely Road, Johnson City, TN 37604   
12  65e5e2dc38f826f2926720

{"cost": 0.1}


I completed the function
               From_OBJECTID                            From_Address  \
0   65e5e58338f826f29267201d  3315 Joshuea Lane, Kingsport, TN 37664   
1   65e5e58338f826f29267201d  3315 Joshuea Lane, Kingsport, TN 37664   
2   65e5e58338f826f29267201d  3315 Joshuea Lane, Kingsport, TN 37664   
3   65e5e58338f826f29267201d  3315 Joshuea Lane, Kingsport, TN 37664   
4   65e5e58338f826f29267201d  3315 Joshuea Lane, Kingsport, TN 37664   
5   65e5e58338f826f29267201d  3315 Joshuea Lane, Kingsport, TN 37664   
6   65e5e58338f826f29267201d  3315 Joshuea Lane, Kingsport, TN 37664   
7   65e5e58338f826f29267201d  3315 Joshuea Lane, Kingsport, TN 37664   
8   65e5e58338f826f29267201d  3315 Joshuea Lane, Kingsport, TN 37664   
9   65e5e58338f826f29267201d  3315 Joshuea Lane, Kingsport, TN 37664   
10  65e5e58338f826f29267201d  3315 Joshuea Lane, Kingsport, TN 37664   
11  65e5e58338f826f29267201d  3315 Joshuea Lane, Kingsport, TN 37664   
12  65e5e58338f826f29267201d  3315 Josh

{"cost": 0.095}


I completed the function
               From_OBJECTID                            From_Address  \
0   65e5e69338f826f29267201f  328 Barnett Drive, Kingsport, TN 37664   
1   65e5e69338f826f29267201f  328 Barnett Drive, Kingsport, TN 37664   
2   65e5e69338f826f29267201f  328 Barnett Drive, Kingsport, TN 37664   
3   65e5e69338f826f29267201f  328 Barnett Drive, Kingsport, TN 37664   
4   65e5e69338f826f29267201f  328 Barnett Drive, Kingsport, TN 37664   
5   65e5e69338f826f29267201f  328 Barnett Drive, Kingsport, TN 37664   
6   65e5e69338f826f29267201f  328 Barnett Drive, Kingsport, TN 37664   
7   65e5e69338f826f29267201f  328 Barnett Drive, Kingsport, TN 37664   
8   65e5e69338f826f29267201f  328 Barnett Drive, Kingsport, TN 37664   
9   65e5e69338f826f29267201f  328 Barnett Drive, Kingsport, TN 37664   
10  65e5e69338f826f29267201f  328 Barnett Drive, Kingsport, TN 37664   
11  65e5e69338f826f29267201f  328 Barnett Drive, Kingsport, TN 37664   
12  65e5e69338f826f29267201f  328 Barne

In [44]:
uniqueDataRemaining = buildData(chunks[17], uniqueDataRemaining, duplicates)

{"geometry": {"x": -9185396.922614545, "y": 4373526.610508647, "spatialReference": {"wkid": 102100, "latestWkid": 3857}}, "attributes": {"FID": 1, "Address": "1429 Miller Street, Kingsport, TN 37664", "Status": "unconnected", "OBJECTID": "65e5e76038f826f292672021", "Role": "child"}}


{"geometry": {"x": -9180050.577136153, "y": 4375493.709649952, "spatialReference": {"wkid": 102100, "latestWkid": 3857}}, "attributes": {"FID": 2, "Address": "120 Shuler Drive, Kingsport, TN 37664", "Status": "unconnected", "OBJECTID": "65e5fcaa38f826f292672022", "Role": "child"}}


{"geometry": {"x": -9181655.754730143, "y": 4378377.362458252, "spatialReference": {"wkid": 102100, "latestWkid": 3857}}, "attributes": {"FID": 3, "Address": "2009 Wandering Drive, Kingsport, TN 37660", "Status": "unconnected", "OBJECTID": "65e5fd0e38f826f292672023", "Role": "child"}}


{"geometry": {"x": -9184413.432850609, "y": 4358473.500314463, "spatialReference": {"wkid": 102100, "latestWkid": 3857}}, "attributes": {"FID"

{"cost": 0.09}


I completed the function
               From_OBJECTID                                 From_Address  \
0   65e5e6f638f826f292672020  136 Chippendale Square, Kingsport, TN 37660   
1   65e5e6f638f826f292672020  136 Chippendale Square, Kingsport, TN 37660   
2   65e5e6f638f826f292672020  136 Chippendale Square, Kingsport, TN 37660   
3   65e5e6f638f826f292672020  136 Chippendale Square, Kingsport, TN 37660   
4   65e5e6f638f826f292672020  136 Chippendale Square, Kingsport, TN 37660   
5   65e5e6f638f826f292672020  136 Chippendale Square, Kingsport, TN 37660   
6   65e5e6f638f826f292672020  136 Chippendale Square, Kingsport, TN 37660   
7   65e5e6f638f826f292672020  136 Chippendale Square, Kingsport, TN 37660   
8   65e5e6f638f826f292672020  136 Chippendale Square, Kingsport, TN 37660   
9   65e5e6f638f826f292672020  136 Chippendale Square, Kingsport, TN 37660   
10  65e5e6f638f826f292672020  136 Chippendale Square, Kingsport, TN 37660   
11  65e5e6f638f826f292672020  136 Chippendale Squar

{"cost": 0.085}


I completed the function
               From_OBJECTID                             From_Address  \
0   65e5e76038f826f292672021  1429 Miller Street, Kingsport, TN 37664   
1   65e5e76038f826f292672021  1429 Miller Street, Kingsport, TN 37664   
2   65e5e76038f826f292672021  1429 Miller Street, Kingsport, TN 37664   
3   65e5e76038f826f292672021  1429 Miller Street, Kingsport, TN 37664   
4   65e5e76038f826f292672021  1429 Miller Street, Kingsport, TN 37664   
5   65e5e76038f826f292672021  1429 Miller Street, Kingsport, TN 37664   
6   65e5e76038f826f292672021  1429 Miller Street, Kingsport, TN 37664   
7   65e5e76038f826f292672021  1429 Miller Street, Kingsport, TN 37664   
8   65e5e76038f826f292672021  1429 Miller Street, Kingsport, TN 37664   
9   65e5e76038f826f292672021  1429 Miller Street, Kingsport, TN 37664   
10  65e5e76038f826f292672021  1429 Miller Street, Kingsport, TN 37664   
11  65e5e76038f826f292672021  1429 Miller Street, Kingsport, TN 37664   
12  65e5e76038f826f2926720

...

I completed the function
               From_OBJECTID                                From_Address  \
0   65e5ffef38f826f292672028  3613 Watterson Street, Kingsport, TN 37660   
1   65e5ffef38f826f292672028  3613 Watterson Street, Kingsport, TN 37660   
2   65e5ffef38f826f292672028  3613 Watterson Street, Kingsport, TN 37660   
3   65e5ffef38f826f292672028  3613 Watterson Street, Kingsport, TN 37660   
4   65e5ffef38f826f292672028  3613 Watterson Street, Kingsport, TN 37660   
5   65e5ffef38f826f292672028  3613 Watterson Street, Kingsport, TN 37660   
6   65e5ffef38f826f292672028  3613 Watterson Street, Kingsport, TN 37660   
7   65e5ffef38f826f292672028  3613 Watterson Street, Kingsport, TN 37660   
8   65e5ffef38f826f292672028  3613 Watterson Street, Kingsport, TN 37660   
9   65e5ffef38f826f292672028  3613 Watterson Street, Kingsport, TN 37660   
10  65e5ffef38f826f292672028  3613 Watterson Street, Kingsport, TN 37660   

   From_Role               To_OBJECTID  \
0      child  65e604

{"cost": 0.05}


I completed the function
              From_OBJECTID                            From_Address From_Role  \
0  65e6004638f826f292672029  1037 Union Street, Kingsport, TN 37660     child   
1  65e6004638f826f292672029  1037 Union Street, Kingsport, TN 37660     child   
2  65e6004638f826f292672029  1037 Union Street, Kingsport, TN 37660     child   
3  65e6004638f826f292672029  1037 Union Street, Kingsport, TN 37660     child   
4  65e6004638f826f292672029  1037 Union Street, Kingsport, TN 37660     child   
5  65e6004638f826f292672029  1037 Union Street, Kingsport, TN 37660     child   
6  65e6004638f826f292672029  1037 Union Street, Kingsport, TN 37660     child   
7  65e6004638f826f292672029  1037 Union Street, Kingsport, TN 37660     child   
8  65e6004638f826f292672029  1037 Union Street, Kingsport, TN 37660     child   
9  65e6004638f826f292672029  1037 Union Street, Kingsport, TN 37660     child   

                To_OBJECTID                                        To_Address  \
0 

{"cost": 0.045}


I completed the function
              From_OBJECTID                            From_Address From_Role  \
0  65e6009738f826f29267202a  417 Church Road, Fall Branch, TN 37656     child   
1  65e6009738f826f29267202a  417 Church Road, Fall Branch, TN 37656     child   
2  65e6009738f826f29267202a  417 Church Road, Fall Branch, TN 37656     child   
3  65e6009738f826f29267202a  417 Church Road, Fall Branch, TN 37656     child   
4  65e6009738f826f29267202a  417 Church Road, Fall Branch, TN 37656     child   
5  65e6009738f826f29267202a  417 Church Road, Fall Branch, TN 37656     child   
6  65e6009738f826f29267202a  417 Church Road, Fall Branch, TN 37656     child   
7  65e6009738f826f29267202a  417 Church Road, Fall Branch, TN 37656     child   
8  65e6009738f826f29267202a  417 Church Road, Fall Branch, TN 37656     child   

                To_OBJECTID                                        To_Address  \
0  65e601b138f826f29267202b          1820 Birchwood Road, Kingsport, TN 37660   
1 

In [45]:
uniqueDataRemaining = buildData(chunks[18], uniqueDataRemaining, duplicates)

{"geometry": {"x": -9183725.499596043, "y": 4365772.188991854, "spatialReference": {"wkid": 102100, "latestWkid": 3857}}, "attributes": {"FID": 1, "Address": "631 Lebanon Road, Kingsport, TN 37663", "Status": "unconnected", "OBJECTID": "65e6021338f826f29267202c", "Role": "child"}}


{"geometry": {"x": -9192836.793367635, "y": 4380367.56223497, "spatialReference": {"wkid": 102100, "latestWkid": 3857}}, "attributes": {"FID": 2, "Address": "2127 Ramsey Avenue, Kingsport, TN 37665", "Status": "unconnected", "OBJECTID": "65e6027b38f826f29267202d", "Role": "child"}}


{"geometry": {"x": -9189321.53936683, "y": 4376896.5850838795, "spatialReference": {"wkid": 102100, "latestWkid": 3857}}, "attributes": {"FID": 3, "Address": "929 Robertson Street, Kingsport, TN 37660", "Status": "unconnected", "OBJECTID": "65e602f838f826f29267202e", "Role": "child"}}


{"geometry": {"x": -9188826.397283606, "y": 4381332.656825469, "spatialReference": {"wkid": 102100, "latestWkid": 3857}}, "attributes": {"FID":

{"cost": 0.04}


I completed the function
              From_OBJECTID                              From_Address  \
0  65e601b138f826f29267202b  1820 Birchwood Road, Kingsport, TN 37660   
1  65e601b138f826f29267202b  1820 Birchwood Road, Kingsport, TN 37660   
2  65e601b138f826f29267202b  1820 Birchwood Road, Kingsport, TN 37660   
3  65e601b138f826f29267202b  1820 Birchwood Road, Kingsport, TN 37660   
4  65e601b138f826f29267202b  1820 Birchwood Road, Kingsport, TN 37660   
5  65e601b138f826f29267202b  1820 Birchwood Road, Kingsport, TN 37660   
6  65e601b138f826f29267202b  1820 Birchwood Road, Kingsport, TN 37660   
7  65e601b138f826f29267202b  1820 Birchwood Road, Kingsport, TN 37660   

  From_Role               To_OBJECTID  \
0     child  65e6021338f826f29267202c   
1     child  65e6027b38f826f29267202d   
2     child  65e602f838f826f29267202e   
3     child  65e6038938f826f29267202f   
4     child  65e603f238f826f292672030   
5     child  65e6044938f826f292672031   
6     child  65e6049b38f826f29

{"cost": 0.035}


I completed the function
              From_OBJECTID                           From_Address From_Role  \
0  65e6021338f826f29267202c  631 Lebanon Road, Kingsport, TN 37663     child   
1  65e6021338f826f29267202c  631 Lebanon Road, Kingsport, TN 37663     child   
2  65e6021338f826f29267202c  631 Lebanon Road, Kingsport, TN 37663     child   
3  65e6021338f826f29267202c  631 Lebanon Road, Kingsport, TN 37663     child   
4  65e6021338f826f29267202c  631 Lebanon Road, Kingsport, TN 37663     child   
5  65e6021338f826f29267202c  631 Lebanon Road, Kingsport, TN 37663     child   
6  65e6021338f826f29267202c  631 Lebanon Road, Kingsport, TN 37663     child   

                To_OBJECTID                                        To_Address  \
0  65e6027b38f826f29267202d           2127 Ramsey Avenue, Kingsport, TN 37665   
1  65e602f838f826f29267202e         929 Robertson Street, Kingsport, TN 37660   
2  65e6038938f826f29267202f             1126 Qualls Road, Kingsport, TN 37660   
3  65e603f

...

{"cost": 0.01}


I completed the function
              From_OBJECTID                           From_Address From_Role  \
0  65e6044938f826f292672031  800 Teasel Drive, Kingsport, TN 37660     child   
1  65e6044938f826f292672031  800 Teasel Drive, Kingsport, TN 37660     child   

                To_OBJECTID                                        To_Address  \
0  65e6049b38f826f292672032        124 Bloomingdale Pike, Kingsport, TN 37660   
1  65ef076606e0a866af56d202  1120 Hubert Taylor Road, Mountain City, TN 37683   

  To_Role  Total_Miles  Total_Minutes  
0   child     0.577778        2.47506  
1   child    71.816849      95.536427  


Database connected



fromRole: child   toRole:child
65e6044938f826f292672031 to 65e6049b38f826f292672032 is inserted in the MySQL database

fromRole: child   toRole:child
65e6044938f826f292672031 to 65ef076606e0a866af56d202 is inserted in the MySQL database
-------------------------------------------------------------------------------------------------------------

{"cost": 0.005}


I completed the function
              From_OBJECTID                                From_Address  \
0  65e6049b38f826f292672032  124 Bloomingdale Pike, Kingsport, TN 37660   

  From_Role               To_OBJECTID  \
0     child  65ef076606e0a866af56d202   

                                         To_Address To_Role  Total_Miles  \
0  1120 Hubert Taylor Road, Mountain City, TN 37683   child    71.325557   

   Total_Minutes  
0      93.561281  


Database connected



fromRole: child   toRole:child
65e6049b38f826f292672032 to 65ef076606e0a866af56d202 is inserted in the MySQL database
---------------------------------------------------------------------------------------------------------------------------------
All data entered into database
//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
////////////////////////////////////////////////////////////////////////////////////////////////

ValueError: Columns must be same length as key